<a href="https://colab.research.google.com/github/DCI-alxogm/ml2023-Isaias-Hernandez/blob/main/Sep29-30/MCMC_Metropoli_30_08_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Monte Carlo Markov Chain Metropolis algorithm

Por Isaias Hernández Lares

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt


In [ ]:
datos = np.loadtxt("/content/drive/MyDrive/ML2023/olympic100m_men.txt", delimiter=",")
datos_x = datos[:, 0]
datos_y = datos[:, 1]
datos_x_norm = (datos_x - np.mean(datos_x))/(np.std(datos_x))
datos_y_norm = (datos_y - np.mean(datos_y))/(np.std(datos_y))

In [ ]:
#Hacer que la pendiente sea negativa
#Modelo
def Modelo(x, theta):
  y = theta[0] + theta[1]*x
  return y

#Para evaluar el likelihood
def Log_Likelihood(x, y, theta, modelo):
  likelihood = np.sum((y - modelo(x, theta))**2)
  return likelihood

#Para evaluar el prior
def Log_Prior(theta, theta_min, theta_max):
  prueba = 1
  for i in range(len(theta)):
    if theta[i] < theta_min[i] or theta[i] > theta_max[i]:
      prueba *= 0
  if prueba == 1:
    prior = 0
  else:
    prior = -np.inf
  return prior

#Para evaluar el posterior
def Log_Posterior(x, y, theta, modelo, theta_min, theta_max):
  posterior = Log_Likelihood(x, y, theta, modelo) + Log_Prior(theta, theta_min, theta_max)
  return posterior

# Definiendo la función del método MCM Metropoli

In [ ]:
def Metropoli(x, y, iteraciones, modelo, theta_min, theta_max, Log_Likelihood, Log_Prior, Log_Posterior):
#Paso 1
  theta_inicial = [32, -0.5]

  #Paso 2
  #Definir cosas
  sigma1 = 0.05
  sigma2 = 0.001
  gaussian_rng = np.random.default_rng()
  registro_aceptadas = []
  #registro_rechazadas = []

  #Ciclo for
  for i in range(iteraciones):
    theta_propuesta = [gaussian_rng.normal(theta_inicial[0], sigma1), gaussian_rng.normal(theta_inicial[1], sigma2)]

  #Paso 3

    posterior_propuesta = Log_Posterior(x, y, theta_propuesta, modelo, theta_min, theta_max)
    posterior_inicial = Log_Posterior(x, y, theta_inicial, modelo, theta_min, theta_max)


    if posterior_propuesta > posterior_inicial:
      theta_inicial = theta_propuesta
      registro_aceptadas.append(theta_propuesta)

    else:
      r = np.log(random.random())
      if (posterior_propuesta - posterior_inicial) > r:
        theta_inicial = theta_propuesta
        registro_aceptadas.append(theta_propuesta)

      else:
        registro_aceptadas.append(theta_inicial)
  return registro_aceptadas

In [ ]:
#Definiendo variables
theta_min = [0, -10]
theta_max = [500, 0]

thetas_buenos = Metropoli(datos_x_norm, datos_y_norm, 1000, Modelo, theta_min, theta_max, Log_Likelihood, Log_Prior, Log_Posterior)
resultados = np.array(thetas_buenos)
plt.scatter(resultados[:, 0], resultados[:,1])
